In [1]:
%mkdir IRDM_data
%cd IRDM_data
%ls

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
 AML_DataScienceBowl  'Colab Notebooks'  'Getting started.pdf'	 IRDM_data   RN
/content/drive/My Drive
mkdir: cannot create directory ‘IRDM_data’: File exists
/content/drive/My Drive/IRDM_data
DataGenerator.py     part2/        query_passage_raw.json  val_idf.json
glove.6B.50d.txt     part2.zip     train_idf.json
gloveEmbedding.json  __pycache__/  utils.py


In [0]:
# %ls
# ! unzip part2.zip -d .
# ! unzip glove.6B.zip -d .
# %ls
# %cd glove.6B
# %ls
# %rm glove.6B.100d.txt  glove.6B.200d.txt glove.6B.300d.txt glove.6B.zip
# %ls

In [0]:
from utils import *
pre_trained_model_path = 'glove.6B.50d.txt'
gloveEmbedding_path = 'gloveEmbedding.json'
# ExtractPreTrained(pre_trained_model_path, gloveEmbedding_path)

In [5]:
import os
import numpy as np
import json
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import tqdm
import csv
import pandas as pd
import time
import re
from nltk.stem import PorterStemmer, SnowballStemmer
from nltk.corpus import stopwords 
import nltk
nltk.download('stopwords')
%tensorflow_version 2.x
import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten, Conv1D, Conv2D, GlobalMaxPool1D, Dropout, Dot
from tensorflow.keras import Model, Input
from DataGenerator import dataPipeLine

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [0]:
embedding_path = 'gloveEmbedding.json'
train_data_path = 'part2/train_data.tsv'
train_idf_path = 'train_idf.json'

test_data_path = 'part2/validation_data.tsv'
test_idf_path = 'val_idf.json'

In [7]:

train_data = dataPipeLine(embedding_path, train_data_path, train_idf_path)
test_data = dataPipeLine(embedding_path, test_data_path, test_idf_path)

num lines  4364339
pos  0.001099135516283222
neg  0.9989008644837167
num lines  1103039
pos  0.00109515620027941
neg  0.9989048437997206


In [0]:
class SimilarityTransform(tf.keras.layers.Layer):

  def __init__(self, input_dim=100):
    super(SimilarityTransform, self).__init__()
    w_init = tf.keras.initializers.GlorotUniform()
    self.w = tf.Variable(initial_value=w_init(shape=(input_dim, input_dim),
                                              dtype='float32'),
                         trainable=True)

  def call(self, x_q):
    return tf.matmul(x_q, self.w)

In [0]:

def RankShortTextCNN(batch_size, max_queryLength, max_passLength, kernel_size):
  
  queryInput = Input(shape=( max_queryLength, 50))
  passInput = Input(shape=(max_passLength, 50))
  ad = Input(shape=(2))


  #query side
  queryConv = Conv1D(100, kernel_size, activation='relu', kernel_regularizer = tf.keras.regularizers.l2(1e-4))(queryInput)#10 x 50 (embedding last) -> 100 x (10-5+1) x 100
  queryConv = GlobalMaxPool1D()(queryConv) # -> 100

  #pass side
  passConv = Conv1D(100, kernel_size, activation='relu', kernel_regularizer = tf.keras.regularizers.l2(1e-4))(passInput)# 50 x50 (embedding last) -> 100 x (50-5+1) x 100
  passConv = GlobalMaxPool1D()(passConv) # -> 100 


  # x_q_prime = M @ x_q
  x_q_prime = SimilarityTransform()(queryConv)
  # print('x_q_prime ', x_q_prime.get_shape().as_list())
  sim = tf.keras.layers.Dot(axes=-1)([x_q_prime, passConv])


  #hidden layer
  
  join_layer = tf.keras.layers.Concatenate(axis=1)([passConv, sim, queryConv, ad])
  hidden_layer = Dense(203, kernel_regularizer = tf.keras.regularizers.l2(1e-4) )(join_layer)
  hidden_layer = tf.keras.layers.Dropout(0.5)(hidden_layer)
  
  out = Dense(1)(hidden_layer)

  model = Model(inputs=[queryInput, passInput, ad], outputs=out)
  return model




In [0]:
@tf.function
def train_step(query, passage, labels, addFeature, sample_weights):

  with tf.GradientTape() as tape:
    prediction = ranknn(inputs=[query, passage, addFeature], training=True)
    loss = loss_object(labels, prediction, sample_weight=sample_weights)
  gradients = tape.gradient(loss, ranknn.trainable_variables)
  optimizer.apply_gradients(zip(gradients, ranknn.trainable_variables))

  train_loss(loss)
  train_accuracy(labels, tf.math.sigmoid(prediction))

@tf.function
def test_step(query, passage, labels, addFeature):
  prediction = ranknn(inputs=[query, passage, addFeature], training=False)
  t_loss = loss_object(labels, tf.math.sigmoid(prediction))

  test_loss(t_loss)
  test_accuracy(labels, prediction)

In [11]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))
#with tf.device(device_name):

with tf.device(device_name):
  tf.keras.backend.clear_session()

  lr = 1e-4
  maxIteration = 5000
  batch_size = 100
  max_queryLength = 10
  max_passageLength = 40
  kernel_size = 5
  size_query = max_queryLength + 2*(kernel_size - 1)
  size_pass = max_passageLength + 2*(kernel_size -1)


  train_dataGenerator = train_data.getItem_tf(max_queryLength, max_passageLength, kernel_size, oversample=False, batch_size=batch_size, wide_conv=True)
  test_dataGenerator = test_data.getItem_tf(max_queryLength, max_passageLength, kernel_size, batch_size=batch_size, oversample=True, wide_conv=True)

  ranknn = RankShortTextCNN(batch_size, size_query, size_pass, kernel_size)
  print(ranknn.summary())

  loss_object = tf.keras.losses.BinaryCrossentropy(from_logits=True)
  # loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
  optimizer = tf.keras.optimizers.Adam(learning_rate=lr)

  train_loss = tf.keras.metrics.Mean(name='train_loss')
  train_accuracy = tf.keras.metrics.BinaryAccuracy(name='train_accuracy')
  test_loss = tf.keras.metrics.Mean(name='test_loss')
  test_accuracy = tf.keras.metrics.BinaryAccuracy(name='test_accuracy')

Found GPU at: /device:GPU:0
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 18, 50)]     0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 48, 50)]     0                                            
__________________________________________________________________________________________________
conv1d (Conv1D)                 (None, 14, 100)      25100       input_1[0][0]                    
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 44, 100)      25100       input_2[0][0]                    
__________________________________________________________________

In [0]:
def weightSample(labels):
  pos_weight = labels.shape[0]/np.sum(labels)
  neg_weight = labels.shape[0]/(labels.shape[0] - np.sum(labels))

  # pos_weight = np.sum(labels)/labels.shape[0]
  # neg_weight = labels.shape[0]/(labels.shape[0] - np.sum(labels))

  sample_weights = labels*pos_weight + (1-labels)*neg_weight
  return sample_weights

In [14]:
with tf.device(device_name):
  # Reset the metrics at the start of the next epoch
  train_loss.reset_states()
  train_accuracy.reset_states()
  test_loss.reset_states()
  test_accuracy.reset_states()

  for i in range(maxIteration):
    train_query, train_passage, train_labels, train_addFeature, = next(train_dataGenerator)
    train_sample_weights = weightSample(train_labels)
    train_step(train_query, train_passage, train_labels, train_addFeature, train_sample_weights)

    if (i+1) % 50 == 0 or i==0:
      test_query, test_passage, test_labels, test_addFeature = next(test_dataGenerator)
      if i == 0:
        print('Train: Generate query {} pass {} label {} ratio check {} feature {}'.format\
              (train_query.shape, train_passage.shape, train_labels.shape, np.sum(train_labels), train_addFeature.shape))
        print('Test: Generate query {} pass {} label {} ratio check {} feature {}'.format\
              (test_query.shape, test_passage.shape, test_labels.shape, np.sum(test_labels), test_addFeature.shape))
      test_step(test_query, test_passage, test_labels, test_addFeature)
      template = 'Iteration {}/{}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
      print(template.format(i+1, maxIteration,
                            train_loss.result(),
                            train_accuracy.result()*100,
                            test_loss.result(),
                            test_accuracy.result()*100))


Train: Generate query (100, 18, 50) pass (100, 48, 50) label (100, 1) ratio check 10.0 feature (100, 2)
Test: Generate query (97, 18, 50) pass (97, 48, 50) label (97, 1) ratio check 47.0 feature (97, 2)
Iteration 1/5000, Loss: 2.498440742492676, Accuracy: 58.999996185302734, Test Loss: 0.7205217480659485, Test Accuracy: 50.51546096801758
Iteration 50/5000, Loss: 1.894934058189392, Accuracy: 46.79999923706055, Test Loss: 0.7256848812103271, Test Accuracy: 48.73096466064453
Iteration 100/5000, Loss: 1.8089728355407715, Accuracy: 48.10481262207031, Test Loss: 0.7260473370552063, Test Accuracy: 51.36054229736328
Iteration 150/5000, Loss: 1.747023105621338, Accuracy: 48.456565856933594, Test Loss: 0.7250466346740723, Test Accuracy: 50.76530456542969
Iteration 200/5000, Loss: 1.6946415901184082, Accuracy: 49.469947814941406, Test Loss: 0.7250713109970093, Test Accuracy: 51.016258239746094
Iteration 250/5000, Loss: 1.6485472917556763, Accuracy: 50.6120491027832, Test Loss: 0.7242075800895691,

In [19]:
#reranking
src_path = './'
raw_path = 'query_passage_raw.json'
with open(src_path + raw_path, 'r') as readFile:
    temp = json.load(readFile)
    query_raw = temp['query']
    labels = temp['query_pass']
readFile.close()
del temp

avg_precision = {}
ndcg = {}

try:
    os.remove('NN.txt')
except OSError:
    pass

for query_id, query_value in zip(query_raw.keys(), query_raw.values()):
    print(query_id, query_value)
    query_pass_pid, query_em, pass_em, addFeature = test_data.getItemByGroup_tf(query_id)
    # query_pass_pid, query_pass_em = Data_Embedding(embedding, val_reader, query_id, val_idf[str(query_id)])
    y_pred = ranknn(inputs=[query_em, pass_em, addFeature], training=False)
    y_pred = tf.math.sigmoid(y_pred).numpy()
    # print(y_pred.shape)
    reranked_candidate = saveToText('NN', query_id, query_pass_pid, y_pred)
    avg_precision[query_id] = avg_Precision(reranked_candidate, labels[query_id])
    ndcg[query_id] = NDCG(query_id, reranked_candidate, labels[query_id])


# print(avg_precision)
# print(ndcg)   

print('Mean value of the average precision is {}'.format(np.mean(list(avg_precision.values()))))
avg_precision['mean'] = np.mean(list(avg_precision.values()))
metricToText(avg_precision, 'NN_Average_Precision')
print('Mean value of NDCG is {}'.format(np.mean(list(ndcg.values()))))
ndcg['mean'] = np.mean(list(ndcg.values()))
metricToText(ndcg, 'NN_NDCG')

1082792 what does the golgi apparatus do to the proteins and lipids once they arrive  
995825 where is the graphic card located in the cpu
1091246 property premises meaning
1047854 what is printing mechanism
991832 who discovered the element carbon
185299 fastest cell phone processor
574730 what are the three monetary policy tools of the fed
1085008 what did maria theresa do for the serfs
609628 what county is mitchell south dakota in
942221 when did the sitcom  rhoda come out 
1001108 where the chromosomes are moving towards the poles of the cell
913568 what type of music is disturbed considered
1056265 what is dry rot answers
432874 java chip frappucino ingredients
1027650 what kind of coding does abantecart use 
1027817 what kind of business is wild planet
1031456 who is city market grocery stores owned by
27743 asus phones price list
1043545 what is the buffer stock model 
1101531 difference between a goal and an expected outcome
1045203 who owns cnn cable news network
995280 which

0